## Creating a custom `gym` environment for the Inventory Management problem Part 3: Implementing `reset()`

- In `__init__()` we defined the allowed ranges for the problem parameters

<img src="images/sample/1.png" width="500"/>

- For a particular shop, the values of `daily_mean_demand`, `unit_selling_price` and `daily_holding_cost_per_unit` is constant.

| Shop Number | Product | $\lambda$ | `unit_selling_price` | `unit_buying_price` | `daily_holding_cost_per_unit` |
| --- | --- | --- | --- | --- | --- |
| 1 | Bananas | 50 | 5 | 2 | 0.5 |
| 2 | Wine | 30 | 15 | 7 | 0.25 |
| ... | ... | ... | ... | ... | ... |

- Each episode simulates a different shop with different parameters. The parameters stay constant during an episode.

| Episode Number | $\lambda$ | `unit_selling_price` | `unit_buying_price` | `daily_holding_cost_per_unit` |
| --- | --- | --- | --- | --- |
| 1 | 50 | 5 | 2 | 0.5 |
| 2 | 30 | 15 | 7 | 0.25 |
| ... | ... | ... | ... | ... |

- Goal: train a policy that works for a wide variety of shops with different parameters.

- `reset()` needs to initialize an episode $\implies$ fix the problem parameters by sampling from the parameter space.

<img src="images/sample/2.png" width="500"/>

In [1]:
import gym
from gym.spaces import Box
import numpy as np
from numpy.random import default_rng


class InventoryEnv(gym.Env):
    def __init__(self):
        """
        Must define self.observation_space and self.action_space here
        """
        
        # Define action space: bounds, space type, shape
        
        # Bound: Shelf space is limited
        self.max_capacity = 4000
        
        # Space type: Better to use Box than Discrete, since Discrete will lead to too many output nodes in the NN
        # Shape: rllib cannot handle scalar actions, so turn it into a numpy array with shape (1,)
        self.action_space = Box(low=np.array([0]), high=np.array([self.max_capacity]))
        
        # Define observation space: bounds, space type, shape
        
        # Shape: The lead time controls the shape of observation space
        self.lead_time = 5
        self.obs_dim = self.lead_time + 4
        
        # Bounds: Define high of the remaining observation space elements
        self.max_mean_daily_demand = 200
        self.max_unit_selling_price = 100
        self.max_daily_holding_cost_per_unit = 5
        
        obs_low = np.zeros((self.obs_dim,))
        obs_high = np.array([self.max_capacity for _ in range(self.lead_time)] +
                            [self.max_mean_daily_demand, self.max_unit_selling_price,
                             self.max_unit_selling_price, self.max_daily_holding_cost_per_unit
                             ]
                            )
        self.observation_space = Box(low=obs_low, high=obs_high)
        
        # The random number generator that will be used throughout the environment
        self.rng = default_rng()
        
        # All instance variables are defined in the __init__() method
        self.current_obs = None

    def reset(self):
        """
        Returns: the observation of the initial state
        Reset the environment to initial state so that a new episode (independent of previous ones) may start
        """
        
        # Sample parameter values from the parameter space
        
        # Set mean daily demand (lambda)
        mean_daily_demand = self.rng.uniform() * self.max_mean_daily_demand
        
        # Set selling price
        selling_price = self.rng.uniform() * self.max_unit_selling_price
            
        # Set buying price: buying price cannot be higher than selling price
        buying_price = self.rng.uniform() * selling_price
        
        # Set daily holding cose per unit: holding cost cannot be higher than buying_price
        daily_holding_cost_per_unit = self.rng.uniform() * min(buying_price, self.max_daily_holding_cost_per_unit)
        
        # Return the first observation
        self.current_obs = np.array([0 for _ in range(self.lead_time)] + 
                                    [mean_daily_demand, selling_price, buying_price, daily_holding_cost_per_unit]
                                    )
        return self.current_obs

    def step(self, action):
        """
        Returns: the next observation, the reward, done and optionally additional info
        """
        raise NotImplementedError

    def render(self, mode="human"):
        """
        Returns: None
        Show the current environment state e.g. the graphical window in `CartPole-v1`
        This method must be implemented, but it is OK to have an empty implementation if rendering is not
        important
        """
        pass

    def close(self):
        """
        Returns: None
        This method is optional. Used to cleanup all resources (threads, graphical windows) etc.
        """
        pass
    
    def seed(self, seed=None):
        """
        Returns: List of seeds
        This method is optional. Used to set seeds for the environment's random number generator for 
        obtaining deterministic behavior
        """
        return

| Observation/Action | Sample | `gym Space` |
| --- | --- | --- |
| (Array of) floating point numbers | `[ 1.2, 3.4, 4.1, 0.9]` | `Box` |
| Integer | 1 | `Discrete` |
| Array of integers | `[2, 1]` | `MultiDiscrete` |

- [Documentation of all `gym` `Space`s](https://www.gymlibrary.dev/api/spaces/) 